# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`CHEM-Data`*

---
*@author: Evan*\
*@date: 2023-06-06*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
isam = xr.open_dataset('F:/Data/Project_anqing/May/COMBINE_ACONC_v54_ISAM_intel_CN3AH_135X138_2023_isam.nc')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [3]:
# 将时间点转换为实际时间
times=pd.date_range('2023-05-01-00','2023-05-15-23',freq='h')
times

DatetimeIndex(['2023-05-01 00:00:00', '2023-05-01 01:00:00',
               '2023-05-01 02:00:00', '2023-05-01 03:00:00',
               '2023-05-01 04:00:00', '2023-05-01 05:00:00',
               '2023-05-01 06:00:00', '2023-05-01 07:00:00',
               '2023-05-01 08:00:00', '2023-05-01 09:00:00',
               ...
               '2023-05-15 14:00:00', '2023-05-15 15:00:00',
               '2023-05-15 16:00:00', '2023-05-15 17:00:00',
               '2023-05-15 18:00:00', '2023-05-15 19:00:00',
               '2023-05-15 20:00:00', '2023-05-15 21:00:00',
               '2023-05-15 22:00:00', '2023-05-15 23:00:00'],
              dtype='datetime64[ns]', length=360, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [5]:
isam

<xarray.Dataset>
Dimensions:  (TSTEP: 384, VAR: 49, DATE-TIME: 2, LAY: 26, ROW: 138, COL: 135)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables: (12/50)
    TFLAG    (TSTEP, VAR, DATE-TIME) int32 ...
    O3_AnQ   (TSTEP, LAY, ROW, COL) float32 ...
    O3_AQI   (TSTEP, LAY, ROW, COL) float32 ...
    O3_AQT   (TSTEP, LAY, ROW, COL) float32 ...
    O3_AQA   (TSTEP, LAY, ROW, COL) float32 ...
    O3_OTH   (TSTEP, LAY, ROW, COL) float32 ...
    ...       ...
    PMI_ICO  (TSTEP, LAY, ROW, COL) float32 ...
    PMJ_ICO  (TSTEP, LAY, ROW, COL) float32 ...
    PMK_ICO  (TSTEP, LAY, ROW, COL) float32 ...
    PMI_BCO  (TSTEP, LAY, ROW, COL) float32 ...
    PMJ_BCO  (TSTEP, LAY, ROW, COL) float32 ...
    PMK_BCO  (TSTEP, LAY, ROW, COL) float32 ...
Attributes: (12/33)
    IOAPI_VERSION:  ioapi-3.2: $Id: init3.F90 185 2020-08-28 16:49:45Z coats ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2023156
    CTIME:          143200
    WDATE:          2023156
    ...             ...
    VGLVLS:         [1.     0.9979 0.9956 0.9931 0.9904 0.9875 0.9844 0.9807 ...
    GDNAM:          CN3AH_135X138   
    UPNAM:          COMBINE         
    VAR-LIST:       O3_AnQ          O3_AQI          O3_AQT          O3_AQA   ...
    FILEDESC:       Src Apportionment file output                            ...
    HISTORY:

In [7]:
days=1 # set spin-up days
dataset1=xr.Dataset(
    data_vars=dict(
        # ! vars from ISAM
        O3_AnQ=(['time','level','y','x'],isam.O3_AnQ[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing','units':'ug m-3'}),
        O3_AQI=(['time','level','y','x'],isam.O3_AQI[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Industry','units':'ug m-3'}),
        O3_AQT=(['time','level','y','x'],isam.O3_AQT[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Trasportation','units':'ug m-3'}),
        O3_AQA=(['time','level','y','x'],isam.O3_AQA[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_OTH=(['time','level','y','x'],isam.O3_OTH[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from OTHER','units':'ug m-3'}),
        O3_ICO=(['time','level','y','x'],isam.O3_ICO[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from ICON','units':'ug m-3'}),
        O3_BCO=(['time','level','y','x'],isam.O3_BCO[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from BCON','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres[:26],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset1

<xarray.Dataset>
Dimensions:    (time: 360, level: 26, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-01 ... 2023-05-15T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 703.9 659.0 616.2
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    O3_AnQ     (time, level, y, x) float32 0.8362 1.204 ... 2.85e-19 2.143e-22
    O3_AQI     (time, level, y, x) float32 0.346 0.4952 ... 2.391e-19 2.143e-22
    O3_AQT     (time, level, y, x) float32 0.4897 0.7092 ... 2.526e-19 2.143e-22
    O3_AQA     (time, level, y, x) float32 0.009863 0.01377 ... 2.143e-22
    O3_OTH     (time, level, y, x) float32 5.233 7.274 ... 4.655e-18 4.381e-21
    O3_ICO     (time, level, y, x) float32 0.09772 0.1355 ... 2.143e-22
    O3_BCO     (time, level, y, x) float32 101.3 102.1 106.7 ... 112.8 112.0
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [8]:
days=1 # set spin-up days
dataset2=xr.Dataset(
    data_vars=dict(
        # ! vars from ISAM
        PM_AnQ=(['time','level','y','x'],isam.PMI_AnQ[days*24-8:-8,:,:,:].data+isam.PMJ_AnQ[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing','units':'ug m-3'}),
        PM_AQI=(['time','level','y','x'],isam.PMI_AQI[days*24-8:-8,:,:,:].data+isam.PMJ_AQI[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing Industry','units':'ug m-3'}),
        PM_AQT=(['time','level','y','x'],isam.PMI_AQT[days*24-8:-8,:,:,:].data+isam.PMJ_AQT[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing Trasportation','units':'ug m-3'}),
        PM_AQA=(['time','level','y','x'],isam.PMI_AQA[days*24-8:-8,:,:,:].data+isam.PMJ_AQA[days*24-8:-8,:,:,:].data,{'long name':'PM25 from Anqing Agricultrue','units':'ug m-3'}),
        PM_OTH=(['time','level','y','x'],isam.PMI_OTH[days*24-8:-8,:,:,:].data+isam.PMJ_OTH[days*24-8:-8,:,:,:].data,{'long name':'PM25 from OTHER','units':'ug m-3'}),
        PM_ICO=(['time','level','y','x'],isam.PMI_ICO[days*24-8:-8,:,:,:].data+isam.PMJ_ICO[days*24-8:-8,:,:,:].data,{'long name':'PM25 from ICON','units':'ug m-3'}),
        PM_BCO=(['time','level','y','x'],isam.PMI_BCO[days*24-8:-8,:,:,:].data+isam.PMJ_BCO[days*24-8:-8,:,:,:].data,{'long name':'PM25 from BCON','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres[:26],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset2

<xarray.Dataset>
Dimensions:    (time: 360, level: 26, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-01 ... 2023-05-15T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 703.9 659.0 616.2
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    PM_AnQ     (time, level, y, x) float32 0.02704 0.03541 ... 2.904e-20
    PM_AQI     (time, level, y, x) float32 0.01122 0.01437 ... 2.422e-20
    PM_AQT     (time, level, y, x) float32 0.01564 0.0204 ... 2.621e-20
    PM_AQA     (time, level, y, x) float32 0.0003284 0.0004059 ... 2.164e-20
    PM_OTH     (time, level, y, x) float32 0.6397 0.747 ... 8.604e-07 2.993e-07
    PM_ICO     (time, level, y, x) float32 0.003205 0.003945 ... 2.164e-20
    PM_BCO     (time, level, y, x) float32 14.97 14.63 13.59 ... 3.209 2.9 2.462
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [9]:
compression=dict(zlib=True,complevel=5)

encoding={var:compression for var in dataset1.data_vars}
dataset1.to_netcdf('D:/Download/May_ISAM_O3-1.nc',encoding=encoding)

encoding={var:compression for var in dataset2.data_vars}
dataset2.to_netcdf('D:/Download/May_ISAM_PM-1.nc',encoding=encoding)